### during this process I've noticed that the Phillies tweets are largely off-topic and will need to be re-collected.

In [1]:
#start with all of the imports
import pandas as pd
import glob
import json
import numpy as np
import pandas as pd
import csv
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral5
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg as df
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import FuncTickFormatter
from bokeh.models import HoverTool

In [2]:
#get the data from the grouped tweets spreadsheet and set the index on the Team column
tweet_counts_df=pd.read_csv("master_csv/grouped_tweets_without_mlb.csv",index_col=0)
tweet_counts_df.head()

,team,tweet_grade,tweet
0,angels,1,5
1,angels,2,12
2,angels,3,39
3,angels,4,38
4,angels,5,47


In [11]:
#get the data from the All Analyzed tweets spreadsheet and set the index on the Team column
tweet_df=pd.read_csv("master_csv/Pride_Graded_tweets_Part_2.csv",index_col='team')
tweet_df.head(2)

,neg,neu,pos,compound_score,Unnamed: 0,Unnamed: 0.1,ID,datetime,has_media,is_reply,...,medias,nbr_favorite,nbr_reply,nbr_retweet,tweet,url,user_id,usernameTweet,polarity,tweet_grade
team,,,,,,,,,,,,,,,,,,,,,
angels,0.0,0.780,0.220,0.4767,0,0,1.004470e+18,6/6/2018 17:00,NaN,False,...,NaN,2,0,0,Hey @Angels we are waiting...signed your loya...,/AarontheBaaron/status/1004467998592196614,26010109.0,AarontheBaaron,"{'neg': 0.0, 'neu': 0.78, 'pos': 0.22, 'compou...",4
angels,0.0,0.832,0.168,0.8585,1,1,1.014170e+18,7/3/2018 11:30,NaN,False,...,NaN,34,2,5,OMG @ANGELS YESSSSS! Finally doing the thing....,/OhItsRicardo_/status/1014169422544580608,19224173.0,OhItsRicardo_,"{'neg': 0.0, 'neu': 0.832, 'pos': 0.168, 'comp...",5


In [12]:
#count the unique twitter handles and how often each has tweeted
tweet_df['usernameTweet'].value_counts(ascending=False)

Phillies           157
BHM_Pirate         131
MLB_News247         72
queerfancystats     40
Athletics           35
PROUDFM             28
Pairsonnalites      26
GYMSPORTSBARNY      26
84TigersDetroit     26
outsports           26
JohnMALHOTRA21      22
YouCanPlayTeam      18
NationalsPride      17
The_Vole            16
AthleticsViews      15
royal_pride123      15
senatorjohn         14
jakeweston2000      14
TD_Canada           14
OutatWrigley        14
mortgage_done       14
KCJB_Pride          13
BlueJays            12
JeffreyLuscombe     11
historiancole       11
Texan__Pride        11
Dodger_Jess83       11
GirlSwagger101      11
FOXSportsFL         10
NowRepair           10
                  ... 
theGoodJen           1
JustBader8           1
AnswerDave           1
MessysoRandom        1
TheAthleticMike      1
erick70115           1
navidschreibt        1
_eli_adorno          1
Blaiserboy           1
TotalTrafficPHL      1
madisonpolice        1
JaMarcosRussell      1
6r3susuki  

In [19]:
#make all the usernames lower case

tweet_df['usernameTweet']=tweet_df['usernameTweet'].astype(str).str.lower()

### Add the Phillies into this list once the tweets have been re-collected. Leaving them in now will skew the data and make it harder to tell if the potential plots are showing anything of value.

In [26]:
#make a subset dataframe of just tweets from the various teams

subset_df = tweet_df[tweet_df['usernameTweet'].isin(['angels','astros','athletics','bluejays','braves', 'brewers','cardinals',
                                                    'cubs','dbacks','dodgers','indians','mariners','marlins','mets','orioles',
                                                    'mariners','nationals','padres','pirates','rangers','raysbaseball','reds','redsox',
                                                    'rockies','royals','sfgiants','tigers','twins','yankees','whitesox'])]

subset_df.head()

,neg,neu,pos,compound_score,Unnamed: 0,Unnamed: 0.1,ID,datetime,has_media,is_reply,...,medias,nbr_favorite,nbr_reply,nbr_retweet,tweet,url,user_id,usernameTweet,polarity,tweet_grade
team,,,,,,,,,,,,,,,,,,,,,
angels,0.148,0.530,0.322,0.5562,4,4,1.052970e+18,10/18/2018 13:01,NaN,False,...,NaN,488,16,90,We're proud to join @MLB in taking a stand ag...,/Angels/status/1052967828909944832,39392910.0,angels,"{'neg': 0.148, 'neu': 0.53, 'pos': 0.322, 'com...",4
angels,0.000,0.865,0.135,0.6239,34,34,1.138870e+18,6/12/2019 14:00,NaN,False,...,NaN,121,8,22,Purchase a Pride Night ticket package and re...,/Angels/status/1138868627183194112,39392910.0,angels,"{'neg': 0.0, 'neu': 0.865, 'pos': 0.135, 'comp...",5
angels,0.150,0.654,0.196,0.0516,132,132,7.891310e+17,10/20/2016 11:46,NaN,False,...,NaN,105,4,39,"On this #SpiritDay , join us in taking a stand...",/Angels/status/789130763841130496,39392910.0,angels,"{'neg': 0.15, 'neu': 0.654, 'pos': 0.196, 'com...",3
angels,0.140,0.683,0.176,0.0000,138,138,9.210510e+17,10/19/2017 12:30,NaN,False,...,NaN,174,4,51,"On this #SpiritDay , the Angels join @MLB in ...",/Angels/status/921051038135631872,39392910.0,angels,"{'neg': 0.14, 'neu': 0.683, 'pos': 0.176, 'com...",3
athletics,0.000,0.802,0.198,0.8122,2,2,1.004500e+18,6/6/2018 18:51,NaN,False,...,NaN,48,1,13,.@Stomper00 had a fun and busy day! He visite...,/Athletics/status/1004496086856187904,19607400.0,athletics,"{'neg': 0.0, 'neu': 0.802, 'pos': 0.198, 'comp...",5


In [27]:
#count the handles again

subset_df['usernameTweet'].value_counts(ascending=False)

athletics       35
bluejays        12
marlins          8
yankees          6
tigers           5
nationals        5
pirates          4
angels           4
raysbaseball     4
reds             4
mariners         4
whitesox         4
cardinals        3
royals           2
cubs             2
padres           2
dbacks           2
sfgiants         2
braves           2
mets             1
brewers          1
orioles          1
redsox           1
Name: usernameTweet, dtype: int64

In [63]:
#build a new dataframe to hold the mean and count of overall tweets plus the number of times the team tweeted
tweet_metrics=pd.DataFrame(tweet_df.groupby('team')['compound_score'].mean())
tweet_metrics['user_tweet_count']=tweet_df.groupby('team')['compound_score'].count()
tweet_metrics['team_tweets']=subset_df['usernameTweet'].value_counts()
tweet_metrics['cir_size']= tweet_metrics['compound_score']*100
tweet_metrics['anchor'] = np.arange(len(tweet_metrics)) +1 #if we don't include the +1 then the first number is 0 and that won't work well with our charting
tweet_metrics.head()

,compound_score,user_tweet_count,team_tweets,cir_size,anchor
team,,,,,
angels,0.318168,141,4.0,31.816809,1
athletics,0.446276,718,35.0,44.627563,2
bluejays,0.533989,660,12.0,53.398879,3
braves,0.407102,300,2.0,40.710167,4
brewers,0.384263,204,1.0,38.426324,5


### now let's see if there is any correlation between the average positive/negative score of the tweets vs how often the team tweeted.  We're going to use Bokeh for our plot so we can easily share it on the web later.



Outstanding tasks for the visuals below:
 * Improve the color palette
 * Additional Styling - alpha, linecolor/edgecolor, etc
 * Add the tick labels to reflect the team name
 * Resize the overall chart
 * discard the at-MLB tweets from the second dataframe
 * make a glyph that is both datasets combined
 

In [49]:
output_notebook()

# Store the data in a ColumnDataSource
tweets_cds = ColumnDataSource(tweet_metrics)

# Specify the selection tools to be made available
select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

# Create the figure
fig = figure(plot_height=400,
             plot_width=600,
             x_axis_label='Team',
             y_axis_label='Mean Compound Score',
             title='Mapping the Correlation Between Frequency of Team Tweets and Overall Positive/Negative score',
             toolbar_location='below',
             tools=select_tools)

# Format the y-axis tick labels as percentages
#fig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')

# Add marker for the most positive tweets
fig.circle(x='anchor',
           y='compound_score',
           source=tweets_cds,
           size='team_tweets',
           color='lightblue',
           line_color='blue',
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           nonselection_alpha=0.3)




# Format the tooltip
tooltips = [
            ('Team: ','@team'),
            ('# of Tweets by Team', '@team_tweets'),
            ('Compound Score', '@compound_score')
            
           ]

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips))

# Visualize
show(fig)

Loading BokehJS ...

In [66]:
 #make another glyph based on the frequency of user tweets and the mean score
    
    
output_notebook()


# Store the data in a ColumnDataSource
tweets_cds = ColumnDataSource(tweet_metrics)


# Specify the selection tools to be made available
select_tools = ['box_select', 'lasso_select', 'poly_select', 'tap', 'reset']

# Create the figure
fig = figure(plot_height=400,
             plot_width=600,
             x_axis_label='Team',
             y_axis_label='Mean Compound Score',
             title='Mapping the Correlation Between Frequency of FAN Tweets and Overall Positive/Negative score',
             toolbar_location='below',
             tools=select_tools)

# Format the y-axis tick labels as percentages
#fig.yaxis[0].formatter = NumeralTickFormatter(format='00.0%')

# Add marker for the most positive tweets
fig.circle(x='anchor',
           y='user_tweet_count',
           source=tweets_cds,
           size='cir_size',
           color='lightblue',
           line_color='blue',
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           nonselection_alpha=0.3)




# Format the tooltip
tooltips = [
            ('Team: ','@team'),
            ('# of Tweets by Team', '@team_tweets'),
            ('# of Tweets by Fans', '@user_tweet_count'),
            ('Compound Score', '@compound_score')
            
           ]

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips))

# Visualize
show(fig)

Loading BokehJS ...